# Mini Curso: Introdução a Otimização Combinatória utilizando a linguagem de programação Julia

## Pacotes essenciais

In [ ]:
using Pkg 
Pkg.add("LightGraphs")
Pkg.add("GLPK")
Pkg.add("LinearAlgebra") 
Pkg.add("JuMP")


using LightGraphs, GLPK, LinearAlgebra, JuMP

## Resolvendo problemas de otimização

Exemplo 1.1 (Wolsey 1998):

$$
    \begin{array}{r}
        \max \quad x_1 + 0.64x_2 \\
        \text { s.t. } \quad 50x_1 + 31x_2 \leq 250 \\
        3x_1 - 2x_2 \geq -4 \\
        x_1, x_2 \geq 0, \text { and interger } \\
    \end{array}
$$

## Resolvendo problemas de otimização: Problema de Atribuição

- $n$ pessoas para executar $n$ tarefas distintas
- Cada pessoa deve executar uma única tarefa
- Algumas pessoas são mais aptas a executar certas tarefas.
- $c_{ij}:$ custo estimado para o indivíduo $i$ executar a tarefa $j$
  
**Objetivo:** atribuir tarefas a pessoas, ao menor custo possível.

In [ ]:
c = [0 20 15 12 13
23 0 23 13 17
18 12 0 11 12
23 32 12 0 22
21 25 26 28 0]

n = 5

In [4]:
model = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

$\displaystyle x_{ij} \in \{0,1\}:$ atribuição da tarefa $j$ a pessoa $i$.

In [ ]:
@variable(model, x[i in 1:n, j in 1:n], Bin)

O custo das atribuições deve ser minimizado:

$$\textrm{min } \sum_{i=1}^n \sum_{j=1}^n c_{ij}x_{ij}$$

In [ ]:
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:n if i != j))

Cada pessoa deve efetuar uma única tarefa:
$$\sum_{j=1}^n x_{ij} = 1 \quad \forall i= 1, \dots, n$$

In [ ]:
@constraint(model, job[j in 1:n], sum(x[i,j] for i in 1:n if i!=j) == 1)

Cada tarefa será efetuada por uma única pessoa:
$$\sum_{i=1}^n x_{ij} = 1 \quad \forall j = 1, \dots, n$$

In [ ]:
#Restrições
@constraint(model, pessoa[i in 1:n], sum(x[i,j] for j in 1:n if i != j) == 1)

In [ ]:
optimize!(model)

In [ ]:
for i in 1:n
    for j in 1:n
        if value(x[i,j]) == 1
            println("Pessoa ", i, " foi atribuída ao trabalho: ", j, " com custo: ", c[i,j])
        end
    end
end
println("O custo ótimo foi: ", objective_value(model))

## Resolvendo problemas de otimização: N-Rainhas

<p align="center">
  <img src="images//n-rainhas.png" />
</p>



![](nquee.png)

In [ ]:
using GLPK
using JuMP
using LinearAlgebra

# N-Queens
N = 4

model = Model(GLPK.Optimizer);

Variável $X_{ij} \in \{0,1\}:$ 1 se uma rainha estiver na posição $(i,j)$, 0 caso contrário. 

In [ ]:
@variable(model, x[i=1:N, j=1:N], Bin)

$$\displaystyle \textrm{ max } \sum_{i = 1} ^{N} \sum_{j=1}^N X_{ij}$$

In [ ]:
@objective(model, Max, sum(x[i,j] for i in 1:N, j in 1:N))

Só podemos ter exatamente uma rainha na horizontal
$$\displaystyle  \sum_{i = 1}^{N} X_{ij} \leq 1 \qquad j = 1, \dots, N$$ 

In [ ]:
@constraint(model, horizontal[j in 1:N], sum(x[i,j] for i in 1:N) <= 1)

Só podemos ter exatamente uma rainha na vertical
$$\displaystyle  \sum_{j = 1}^{N} X_{ij} \leq 1 \qquad i = 1, \dots, N$$ 

In [ ]:
@constraint(model, vertical[i in 1:N], sum(x[i,j] for j in 1:N) <= 1)

Só podemos ter exatamente uma rainha na diagonal positiva 
$$\displaystyle  \underbrace{\sum_{i = 1}^{N} \sum_{j = 1}^{N}}_{i - j = k}  X_{ij} \leq 1 \qquad k = -(N-2), \dots, N-2 \quad $$ 

In [ ]:
@constraint(model, diag_pos[k in -(N-2):(N-2)], sum(x[i,j] for i in 1:N, j in 1:N if (i-j) == k) <= 1)

Só podemos ter exatamente uma rainha na diagonal negativa

$$\displaystyle  \underbrace{\sum_{i = 1}^{N} \sum_{j = 1}^{N}}_{i + j = k}  X_{ij} \leq 1 \qquad k = 3, \dots, 2N-1 \quad $$ 

In [ ]:
@constraint(model, diag_neg[k in 3:2*(N-1)], sum(x[i,j] for i in 1:N, j in 1:N if (i + j) == k) <= 1)

In [ ]:
optimize!(model)

In [ ]:
println("Número de rainhas: ", objective_value(model))
solution = convert.(Int,value.(x))

4×4 Array{Int64,2}:
 0  1  0  0
 0  0  0  1
 1  0  0  0
 0  0  1  0

![](images\\sol-8-queens.png)

## Resolvendo problemas de otimização: Problema da Mochila 0-1

### Agora é com você. Desafio!

  - Temos uma mochila com uma capacidade máxima (peso máximo) dado por $W$
  - Imagine que temos $n$ possíveis itens que podem entrar na mochila
  - Cada item $i$ possui um peso $a_i$ específico e seu preço é $c_i$.

**Objetivo: Selecionar um subconjunto de itens que dê o máximo retorno possível, isto é, que o custo total do que está dentro da mochila seja o maior possível!**

**Variáveis**

$x_i \in \{0,1\}:$ pegar ou não o item $i$ para colocar na mochila

**Função Objetivo**

$$\textrm{max } \sum_{i=1}^n c_ix_i$$

**Restrições**

- Não é possível colocar mais itens do que a capacidade $W$ dada.

$$\sum_{i=1}^n a_ix_i \leq W$$

$$x_i \in \{0,1\}$$

In [ ]:
n = 5 # Número de itens 
W = 50 # Peso máximo que a mochila pode carregar

a = [20 12 17 13 17] # Capacidade de cada item
c = [30 11 20 18 17] # Custo de cada item 